In [1]:
# for google colab
from google.colab import drive
# mount your Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
! pip install tenseal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.8 MB/s 


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
import tenseal as ts
import pickle

### Define the Module

In [4]:
class MLP(nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.input_fc = nn.Linear(input_dim, 250)
    self.hidden_fc = nn.Linear(250, 100)
    self.output_fc = nn.Linear(100, output_dim)
  def forward(self, x):
    # x = [batch size, height, width]
    batch_size = x.shape[0]
    x = x.view(batch_size, -1)
    # x = [batch size, height * width]
    h_1 = F.relu(self.input_fc(x))
    # h_1 = [batch size, 250]
    h_2 = F.relu(self.hidden_fc(h_1))
    # h_2 = [batch size, 100]
    y_pred = self.output_fc(h_2)
    # y_pred = [batch size, output dim]
    return y_pred

### Download & Load MNIST Data

In [5]:
EPOCHS = 10 
BATCH_SIZE = 50
DOWNLOAD_MNIST = True

train_data = datasets.MNIST(root = './data', train = True, download = DOWNLOAD_MNIST, transform = transforms.ToTensor())
test_data = datasets.MNIST(root = './data', train = False, download = DOWNLOAD_MNIST, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, num_workers = 0)
test_loader = torch.utils.data.DataLoader(test_data, shuffle = False, num_workers = 0)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



### Train the Module

In [6]:
def train(model):
  lossfunc = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
  for epoch in range(1, EPOCHS+1):
    train_loss = 0.0
    for data,target in train_loader:
      optimizer.zero_grad()
      output = model(data)
      loss = lossfunc(output,target)
      loss.backward()
      optimizer.step()
      train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

model_q2 = MLP(input_dim = 784, output_dim = 10)
train(model_q2)

Epoch: 1 	Training Loss: 0.287934
Epoch: 2 	Training Loss: 0.113236
Epoch: 3 	Training Loss: 0.073971
Epoch: 4 	Training Loss: 0.050303
Epoch: 5 	Training Loss: 0.036536
Epoch: 6 	Training Loss: 0.028867
Epoch: 7 	Training Loss: 0.022894
Epoch: 8 	Training Loss: 0.019612
Epoch: 9 	Training Loss: 0.018482
Epoch: 10 	Training Loss: 0.014636


### Question 2

In [38]:
def test(model):
  correct = 0
  total = 0
  result_q2 = []
  with torch.no_grad(): 
    for images, labels in test_loader:
      outputs = model(images)
      _,predicted = torch.max(outputs.data,1)
      result_q2.append(predicted)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
  return result_q2

result_q2 = test(model_q2)
print(result_q2)
#result_q2 = list(map(int, result))

Accuracy of the network on the test images: 97 %
[tensor([7]), tensor([2]), tensor([1]), tensor([0]), tensor([4]), tensor([1]), tensor([4]), tensor([9]), tensor([5]), tensor([9]), tensor([0]), tensor([6]), tensor([9]), tensor([0]), tensor([1]), tensor([5]), tensor([9]), tensor([7]), tensor([3]), tensor([4]), tensor([9]), tensor([6]), tensor([6]), tensor([5]), tensor([4]), tensor([0]), tensor([7]), tensor([4]), tensor([0]), tensor([1]), tensor([3]), tensor([1]), tensor([3]), tensor([4]), tensor([7]), tensor([2]), tensor([7]), tensor([1]), tensor([2]), tensor([1]), tensor([1]), tensor([7]), tensor([4]), tensor([2]), tensor([3]), tensor([5]), tensor([1]), tensor([2]), tensor([4]), tensor([4]), tensor([6]), tensor([3]), tensor([5]), tensor([5]), tensor([6]), tensor([0]), tensor([4]), tensor([1]), tensor([9]), tensor([5]), tensor([7]), tensor([8]), tensor([9]), tensor([3]), tensor([7]), tensor([4]), tensor([6]), tensor([4]), tensor([3]), tensor([0]), tensor([7]), tensor([0]), tensor([2]), t

### Question 3

In [33]:
# define model
model_q3 = MLP(input_dim = 784, output_dim = 10)
model_q3.load_state_dict(torch.load('./gdrive/MyDrive/AI_HW2/model.pt'))

# load images
dataset = datasets.ImageFolder('./gdrive/MyDrive/AI_HW2/', 
  transform = transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))
dataset_loader = torch.utils.data.DataLoader(dataset, shuffle = False, num_workers = 0)

# test 
result_q3 = []
with torch.no_grad(): 
    for images, labels in dataset_loader:
      outputs = model_q3(images)
      result_q3 += outputs.data[0]

#result_q3 = list(map(float, result))
#print(result_q3)

[tensor(-4.1410), tensor(0.9430), tensor(-0.8795), tensor(4.9334), tensor(-2.7162), tensor(0.0687), tensor(-6.9404), tensor(-0.6003), tensor(-0.1572), tensor(0.5850), tensor(-2.2589), tensor(-0.7217), tensor(1.1939), tensor(0.2018), tensor(-3.4748), tensor(-4.8974), tensor(-6.8439), tensor(4.4236), tensor(-2.2708), tensor(0.6216), tensor(0.5651), tensor(-2.8667), tensor(-1.9500), tensor(-1.8516), tensor(0.2365), tensor(-0.5466), tensor(3.1973), tensor(-2.0847), tensor(0.1094), tensor(0.0586), tensor(-4.1712), tensor(0.7529), tensor(-1.7217), tensor(-0.1344), tensor(0.0229), tensor(-3.8108), tensor(-6.9081), tensor(1.2869), tensor(-0.7755), tensor(2.8409), tensor(-1.4586), tensor(-0.7105), tensor(-1.6464), tensor(-0.5042), tensor(-1.0784), tensor(2.2804), tensor(-0.2068), tensor(-1.2050), tensor(1.0692), tensor(-0.0436)]


### Encrypt results

In [34]:
# q2
context_q2 = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=4096, plain_modulus=1032193)
encrypted_q2 = ts.bfv_vector(context_q2, result_q2)

# q3
context_q3 = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=8192,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
          )
context_q3.global_scale = 2**40
encrypted_q3 = ts.ckks_vector(context_q3, result_q3)

# package results
result = {
  'q2_key': context_q2.serialize(save_secret_key=True),
  'q2_result': encrypted_q2.serialize(),
  'q3_key': context_q3.serialize(save_secret_key=True),
  'q3_result': encrypted_q3.serialize()
}

with open('./gdrive/MyDrive/R11922138_a2.pkl', 'wb') as f:
    pickle.dump(result, f)

The following operations are disabled in this setup: matmul, matmul_plain, conv2d_im2col, replicate_first_slot.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.


/usr/local/lib/python3.7/dist-packages/tenseal/tensors/plaintensor.py:28: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  t = np.array(tensor, dtype=dtype)


In [47]:
with open('./gdrive/MyDrive/R11922138_a2.pkl', 'rb') as f:
    given = pickle.load(f)
context = ts.context_from(given['q3_key'])
#print(ts.ckks_vector_from(context, given['q3_result']).decrypt())
test = list(map(float, result_q3))
temp3 = []
for i in range(len(test)):
      if abs(ts.ckks_vector_from(context, given['q3_result']).decrypt()[i] - test[i]) >= 0.01:
        temp3.append('f')
print(len(temp3))

0
